In [ ]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 55.6 MB/s 
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/lsb-release:DISTRIB_DESCRIPTION="Ubuntu 18.04.6 LTS"
/etc/os-release:NAME="Ubuntu"
/etc/os-release:ID=ubuntu
/etc/os-release:ID_LIKE=debian
/e

In [ ]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-9284qhct
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-9284qhct
     |████████████████████████████████| 671.4 MB 1.7 kB/s 
     |████████████████████████████████| 463 kB 11.2 MB/s 
     |████████████████████████████████| 1.3 MB 47.9 MB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268638 sha256=f8d1205ab8158d0976545ae7511f3371ef7cf33e226845ec1c83f9ca947c4c0f
  Stored in directory: /tmp/pip-ephem-wheel-cache-qaj8e79d/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
Successfully built pykospacing
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempt

In [ ]:
from konlpy.tag import Mecab
from pykospacing import Spacing
from itertools import permutations, combinations, product
import pickle
import numpy as np

mecab = Mecab()
spacing = Spacing()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

with open("/content/gdrive/MyDrive/NLP/real_backup6.p", "rb") as file:  # 학습된 파일 위치
  trained_model=pickle.load(file)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 단어 입력하기

In [ ]:
# input_words = ['구름빵', '홍대와 건대사이', '파란커피']
# input_words = ['임진왜란', '무신정변', '병자호란', '훈민정음 반포', '계유정란']
# input_words = ['의', '의', '의', '의']
input_words = ['김천', '대구', '부산', '서귀포', '원주', '진천']

## 줄임말 추출

In [ ]:
def characters_extract(words):

  # splited_words = []
  # for word in words:
  #   splited_words.append(word.split())

  # characters = []
  # for word in splited_words:
  #   if len(word) == 1:
  #     characters.append(word[0][0])
  #   else:
  #     characters.append([w[0] for w in word])

  # characters_permutation = list(product(*characters))

  # return characters_permutation

  first = []
  for word in words:
    first.append(word[0])
  
  return [first]


def words_extract(characters): # input: 글자 리스트

  windows = [[], characters]  # windows 에는 모든 경우 포함
  for i in range(2, 5):
    permutation = list(permutations(characters, i))
    for j, p in enumerate(permutation):
      concat = ''
      for character in p:
        concat += character
        permutation[j] = concat
    windows.append(permutation)

  is_noun = [[] for _ in range(5)]  # is_noun 에는 단어만 포함
  for i in range(1, 5):
    for w in windows[i]:
      if len(mecab.pos(w)) == 1:
        # is_noun[i].append(w)
        if mecab.pos(w)[0][1][0] in 'NVM':  # 명사, 동사, 형용사, 관형사, 부사
          is_noun[i].append(w)  # index 가 window_size 를 의미함

  return is_noun  # output: window_size 별 조합 단어 리스트


def words_combination(is_noun, characters): # input: 조합 단어 리스트, 글자 리스트

  is_noun_concat = []
  for _ in is_noun:
    is_noun_concat.extend(_)

  combination = []
  for i in range(1, len(characters) + 1):
    combination.extend(list(combinations(is_noun_concat, i)))

  final_combination = []
  for c in combination:
    concat = ''
    for w in c:
      concat += w
    if len(concat) == len(characters):
      flag = 1
      for f in characters:
        if concat.count(f) != characters.count(f):
          flag = 0
          break
      if flag:
        final_combination.append(c)
  
  final_combination = list(set(final_combination))

  return final_combination  # output: 최종 조합 리스트

In [ ]:
def words2abb(words):

  output = []

  characters_permutation = characters_extract(input_words)

  while not output:
    if not characters_permutation:
      break
    
    characters = characters_permutation.pop(0)
    words_by_windows = words_extract(characters)
    output = words_combination(words_by_windows, characters)
    
  return output

In [ ]:
abb = words2abb(input_words)
abb

[('진', '부서', '김대원'),
 ('김', '원대', '부서진'),
 ('김', '서', '대원', '진부'),
 ('김', '부', '원', '서대진'),
 ('김', '부', '서진', '원대'),
 ('서', '대부', '김원진'),
 ('김', '부', '원', '서진대'),
 ('김서진', '부대원'),
 ('서', '원부', '김대진'),
 ('김', '부', '원', '진', '대서'),
 ('김진', '대원', '서부'),
 ('김진', '대원', '부서'),
 ('대', '서원', '김진부'),
 ('김원', '부진', '서대'),
 ('부', '서', '진', '김대원'),
 ('부', '원', '김대', '서진'),
 ('김', '부', '진', '서대원'),
 ('대', '부', '김원', '서진'),
 ('대', '원부', '김진서'),
 ('대', '부진', '김서원'),
 ('김', '부', '대진', '원서'),
 ('부', '진', '김원', '대서'),
 ('원', '부서', '김대진'),
 ('원', '서부', '김대진'),
 ('부', '서대', '김원진'),
 ('부', '원대', '김서진'),
 ('김', '대', '원', '부서진'),
 ('부', '진서', '김원대'),
 ('김', '대', '서원', '진부'),
 ('부', '서', '김원', '대진'),
 ('대', '부', '진', '김서원'),
 ('김', '대', '부진', '원서'),
 ('원', '김서', '부대진'),
 ('김', '부', '대원', '서진'),
 ('대', '진', '김서', '부원'),
 ('서', '진', '김원', '부대'),
 ('김', '서', '원', '진', '대부'),
 ('대', '부서', '김원진'),
 ('김', '원진', '대부서'),
 ('부', '진', '김서', '원대'),
 ('김서', '부대', '원진'),
 ('서', '진', '김대', '부원'),
 ('김', '대', '원서', '진부'),
 

## 문장 생성

In [ ]:
def abb2sent(combs, model):  # input: window_size 별 조합 단어 리스트
  
  made = ['']
  prob = [0]

  for t in combs:
    per = list(permutations(t))
    for k in range(len(per)):
      input = per[k]
      n = len(input)
      sentence = ""
      pp = ['은', '는', '이', '가', '을', '를', '와', '과', '다', '고', '의', '에', '로', '며', '만', '도']  # 조사/어미 리스트
      pp_cnt = 0
      flag = 1
      total_freq = model.unigrams.freq(input[0])
      for i in range(n-1):

        if model[[input[i]]]:  # corpus 에 단어가 존재하는 경우

          if model[[input[i]]][input[i+1]]:  # 두 단어 bigram 으로 바로 연결되는 경우
            sentence += input[i] + " "
            total_freq *= model[[input[i]]].freq(input[i+1])

          else:  # 두 단어가 bigram 으로 연결되지 않고 조사/어미를 활용해 trigram 시도
            pre_freq = []
            post_freq = []
            for p in pp:
              pre_freq.append(model[[input[i]]].freq(p))
              post_freq.append(model[[p]].freq(input[i+1]))
            con_freq = np.array(pre_freq) * np.array(post_freq)
            if max(con_freq):  # trigram 으로 가능한 경우
              sentence += input[i] + pp[con_freq.argmax()] + " "
              total_freq *= max(con_freq)
              pp_cnt += 1
            else:  # trigram 으로도 불가능한 경우
              flag = 0
              break

        else:  # corpus 에 단어가 존재하지 않는 경우
          flag = 0
          break
          
      if flag:  # 해당 조합으로는 문장을 만들 수 없는 경우
        sentence += input[-1]
        made.append(sentence)
        prob.append((total_freq)**(1/(n + pp_cnt)))

      else:  # 문장이 만들어진 경우
        made.append("")
        prob.append(0)
  
  return made, prob  # output: 만들어진 문장과 각각 문장의 확률

In [ ]:
sents, probs = abb2sent(abb, trained_model)
if len(sents) > 1:
  for i in range(1, len(sents)):
    print(sents[i], probs[i])
else:
  print(sents[0], probs[0])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
대 부 원 서 김진 9.711012057551176e-05
대 부 원의 김진은 서 0.0006296007458268187
대 부는 김진은 서 원 0.0007300651426081001
대 부는 김진은 원 서 0.0007655425195605
대 서 부 원의 김진 0.00023503990770573524
대 서 부는 김진은 원 0.0008262353291951012
대 서 원 부는 김진 0.0002349576496893218
대 서 원의 김진은 부 0.0007169745420698988
대 서 김진은 부 원 0.00042048415028098486
대 서 김진은 원 부 0.00040543401968307747
대 원 부 서 김진 6.237668716859774e-05
대 원 부는 김진은 서 0.0006378770465369913
대 원 서 부는 김진 0.0002595770240176161
대 원 서 김진은 부 0.000419857785445644
대 원의 김진은 부 서 0.0005280835675213496
대 원의 김진은 서 부 0.0006786482708184574
대의 김진은 부 서 원 0.000561781008385527
대의 김진은 부 원 서 0.0006733769762409782
대의 김진은 서 부 원 0.0006787841239660644
대의 김진은 서 원 부 0.0006224184608790605
대의 김진은 원 부 서 0.0005119428583871142
대의 김진은 원 서 부 0.0007570365543332924
부 대 서 원의 김진 0.00033302029686890566
부 대 서 김진은 원 0.0006355845946203117
부 대 원 서 김진 0.00015030805944914245
부 대 원의 김진은 서 0.0008601573998915826
부 대의 김진은 서 원 0.0009150447414022788
부 대의 김진은 원 서 0.000959511166827093

## 최종 코드

In [ ]:
def words2sent(words, model):  # input: 줄임말을 생성하고 싶은 단어 리스트

  abb = words2abb(words)
  sents, prob = abb2sent(abb, model)

  best10 = []
  for _ in range(10):
    if max(prob):
      index = np.array(prob).argmax()
      best10.append(((sents.pop(index)), round(np.sqrt(1/prob.pop(index)), 3)))
    else:
      break

  return best10  # output: 최대 10개의 가장 자연스러운 문장

## 결과

In [ ]:
output = words2sent(input_words, trained_model)
output

[('부서와 대의 진원은 김', 18.589),
 ('대는 부서의 진원은 김', 20.84),
 ('원 대의 서진은 김 부', 21.543),
 ('서도 진원은 김 부 대', 21.989),
 ('원의 김진은 부서와 대', 22.199),
 ('진의 부만 원대에 서 김', 22.643),
 ('원의 서진은 김 부 대', 22.747),
 ('부진에 대의 원서는 김', 23.293),
 ('서 부 대의 진원은 김', 23.318),
 ('원 대 부는 진에 서 김', 23.345)]